In [1]:
#Import Statements
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import time

In [2]:


#from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
#from selenium import webdriver
#firefox_binary = FirefoxBinary()
#driver = webdriver.Firefox(firefox_binary=firefox_binary)

#driver = webdriver.Firefox()
#driver.get("http://www.python.org")
#assert "Python" in driver.title
#elem = driver.find_element(By.NAME, "q")
#elem.clear()
#elem.send_keys("pycon")
#elem.send_keys(Keys.RETURN)
#assert "No results found." not in driver.page_source
#driver.close()

In [3]:
#Load Rochelle Data
rochelle = pd.read_excel(r"C:\Users\justi\Downloads\GLANSIS Spp. Tracking.xlsx", sheet_name = "Listed")
#scraped_data_frame = pd.DataFrame() !!!LETS CREATE THE DATA FRAME 
#rochelle.head


In [4]:
#rochelle.columns

In [5]:
#rochelle.reset_index(inplace = True,drop = True) 
#name = rochelle.loc[2]["Scientific Name"] #for some reason couldn't access rochelle[0]...
#print(name)


In [6]:
# Ignore SSL certificate errors --from Beautiful Soup SI 206 textbook
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [7]:
#DEVELOPMENT EXPLORING --- quick learning about using BS with Herbarium
# url = "https://quod.lib.umich.edu/h/herb00ic/x-1000138/MICH-V-1000138A?lasttype=boolean;lastview=reslist;op2=And;q2=Phragmites+australis+australis;resnum=1;rgn2=ic_all;select2=all;size=50;start=1;subview=detail;view=entry"
#html = urllib.request.urlopen(url, context=ctx).read()
#soup = BeautifulSoup(html, 'html.parser')
#soup

In [8]:
#DEVELOPMENT EXPLORING --- quick learning about using BS with Herbarium
#resp = requests.get(url)
#newsoup = BeautifulSoup(resp.content, 'html.parser')
#iteminfo = soup.find('dl')
#print(iteminfo)

In [9]:
#accessing search page of the herbarium
herbarium_url = "https://quod.lib.umich.edu/cgi/i/image/image-idx?xc=1;page=searchgroup;g=herb-ic"
herbarium_html = urllib.request.urlopen(herbarium_url, context=ctx).read()
#herbarium_html


In [10]:
#setting up driver for Selenium
driver = webdriver.Firefox() #???Should I write a script for multiple Browsers?

#send driver to the herbarium website and submit a search query and scrape the amount of results
#BE SURE TO SWITCH ALL "FIND ELEMENTS" TO XPATH IF PLAUSIBLE?
driver.get(herbarium_url)
elem = driver.find_element(by = By.NAME, value = 'q1') #Q1 is the first search box?
elem.send_keys('Cabomba caroliniana') #Sends in a name from Rochelle's list
searchbutton = driver.find_element(by = By.CSS_SELECTOR, value= '#collection-search > button.\[.button.button--cta.\]') #selects the advanced search button based off the pulled tag from right clicking the inspected element
searchbutton.click()
#search_results_url = driver.current_url
resultcount = driver.find_element(by = By.CLASS_NAME, value ='results-heading').text
#next line returns a list of two numbers like ['23','23']
resultcountnumberlist= list(map(int, list(filter(lambda x: x!= '', re.findall(r'\d*', re.search(r'\d* of \d*', resultcount).group()))))) #first number in list is the total results displayed on the page, the second number is the total results
resultitem = driver.find_element(by=By.XPATH, value='/html/body/main/div[2]/div[2]/section[1]/a')
resultitem.click()

#individually select each search result and scrape each matching entry

In [11]:
#driver2 = webdriver.Chrome()
scraped_dataframe = pd.DataFrame(index=[0])
for result in range(int(resultcountnumberlist[0]))[:3]: #[:-1]: #properly traverses the amount of results on the page
    scrapeddata1 = []
    scrapeddata2 = []
    time.sleep(2)
    resp = requests.get(driver.current_url)
    #print(resp.content)
    newsoup = BeautifulSoup(resp.content, 'html.parser')
    collection_category = newsoup.find_all('dt')
    collection_data = newsoup.find_all('dd')
    for tag in collection_category:
        scrapeddata1.append(tag.text)
    for tag in collection_data:
        scrapeddata2.append(tag.text)
    nextbutton = driver.find_element(by=By.XPATH, value='/html/body/main/div[1]/p/a[2]')
    nextbutton.click()
    scraped_data = dict(zip(scrapeddata1, scrapeddata2))
    result_df = pd.DataFrame(data=scraped_data, index=[0])
    if scraped_dataframe.empty:
        scraped_dataframe = result_df
        print('Empty')
    else:
        scraped_dataframe.append(pd.DataFrame(data = result_df.values))
        print('Not Empty Anymore')
    #print(result_df)
#scraped_dataframe.append(result_df)



Empty
Not Empty Anymore
Not Empty Anymore


In [12]:
scraped_dataframe

,Decimal Latitude,Decimal Longitude,Catalog Number,Scientific Name,Family,Identified By,Collector,Collector Number,Event Date,Continent,...,Modified,ID,Occurrence ID,Collection,Image Size,File Size,Record,Link to this Item,Manifest,Full citation
0,44.6617400000,-77.9174900000,1201461,\nCabomba caroliniana\n,Cabombaceae,Michael J. Oldham,Michael J. Oldham,22526,1999-07-19,North America,...,2016-3-11,df39a3bd-695e-4d5b-b21e-5586374c08f8,df39a3bd-695e-4d5b-b21e-5586374c08f8,\nUniversity of Michigan Herbarium Catalog Col...,3753 x 5634,725 KB,1201461,\n\nhttps://quod.lib.umich.edu/h/herb00ic/x-12...,\n\nhttps://quod.lib.umich.edu/cgi/i/image/api...,"\n\n\n""Cabomba caroliniana"". \nhttps://quod.li..."


In [13]:
result_df.values

array([['42.7776300000', '-83.5845300000', '1302380',
        '\nCabomba caroliniana\n', 'Cabombaceae', 'Anton A. Reznicek',
        '10329', '1996-09-04', 'North America', 'USA', 'Michigan',
        'Oakland',
        'Rattelee Lake, SW 1/4 section 1, T4N R7E, ca. 2 1/2 mi. SE of Holly',
        'Shallow water of marly lake.', 'Angiosperms', 'MICH',
        '2016-3-10', '172d7185-adbe-483b-b5c1-67be461d43b0',
        '172d7185-adbe-483b-b5c1-67be461d43b0',
        '\nUniversity of Michigan Herbarium Catalog Collection\n',
        '3744 x 5616', '794 KB', '1302380',
        '\n\nhttps://quod.lib.umich.edu/h/herb00ic/x-1302380/mich-v-1302380\n\ncontent_copy\n\n\n',
        '\n\nhttps://quod.lib.umich.edu/cgi/i/image/api/manifest/herb00ic:1302380:MICH-V-1302380\n\ncontent_copy\n\n\n',
        '\n\n\n"Cabomba caroliniana". \nhttps://quod.lib.umich.edu/h/herb00ic/x-1302380/mich-v-1302380.\n University of Michigan Library Digital Collections. Accessed February 06, 2023.\n\n\ncontent_copy\n\